# import package

In [2]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

# import datasets

## about age

In [4]:
with urlopen('https://opendata.arcgis.com/datasets/a7887f1940b34bf5a02c6f7f27a5cb2c_0.geojson') as f:
    data = json.load(f)

#make empty dictionary and dataframe
to_make_dataframe = {}
df = pd.DataFrame(columns = data['features'][0]['properties'].keys())

#retrieve data from geojson documentation
for j in range(len(data['features'])):
    for k in data['features'][j]['properties'].keys():
            to_make_dataframe[k] = data['features'][j]['properties'][k]
    df_temp = pd.DataFrame([to_make_dataframe])
    df = pd.concat([df,df_temp],axis = 0,ignore_index=True)

#pay attention to the last row, the countyname is unknown, so I drop the last row
df.drop([67], inplace = True)
df = df.sort_values(by = 'COUNTY')

df.keys()

retrieve_list = ['County_1','PUIsTotal', 'Age_0_4', 'Age_5_14', 'Age_15_24',
       'Age_25_34', 'Age_35_44', 'Age_45_54', 'Age_55_64', 'Age_65_74',
       'Age_75_84', 'Age_85plus','TPositive', 'TNegative','C_Female', 'C_Male',
       'C_SexUnkn', 'C_AllResTypes', 'C_Age_0_4', 'C_Age_5_14', 'C_Age_15_24',
       'C_Age_25_34', 'C_Age_35_44', 'C_Age_45_54', 'C_Age_55_64',
       'C_Age_65_74', 'C_Age_75_84', 'C_Age_85plus','T_total', 'T_negative', 'T_positive', 'Deaths']

age_df = df.loc[:,retrieve_list].reset_index(drop = True).rename(columns = {'County_1': "County"})
age_df

,County,PUIsTotal,Age_0_4,Age_5_14,Age_15_24,Age_25_34,Age_35_44,Age_45_54,Age_55_64,Age_65_74,...,C_Age_35_44,C_Age_45_54,C_Age_55_64,C_Age_65_74,C_Age_75_84,C_Age_85plus,T_total,T_negative,T_positive,Deaths
0,Alachua,16394,220,237,2069,3072,2345,2307,2628,1938,...,161,158,126,101,37,19,45208,43619,1578,12
1,Baker,965,30,18,109,145,134,157,159,109,...,15,14,17,10,9,3,2035,1933,102,4
2,Bay,3727,46,47,312,507,523,604,677,471,...,130,112,84,45,21,4,8888,8162,713,5
3,Bradford,1521,23,30,127,200,176,225,294,190,...,14,10,8,7,12,11,2804,2707,95,2
4,Brevard,21386,277,401,1521,2305,2383,2975,4621,4040,...,387,346,311,157,77,49,51772,49242,2521,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,1219,11,26,74,82,112,141,407,279,...,4,5,25,13,2,0,2432,2347,85,2
63,Volusia,22158,196,353,1425,2397,2451,3179,4468,4118,...,374,384,371,284,120,100,43776,41061,2685,58
64,Wakulla,1318,16,31,108,150,178,221,247,218,...,14,16,7,6,3,1,2464,2373,87,1
65,Walton,1832,53,51,137,201,243,269,322,258,...,56,40,59,29,18,6,3972,3650,318,9


## about medical resources

In [9]:
ven_path = 'https://raw.githubusercontent.com/Archerontheway/Summer_C_project/master/Data%20Source/Raw_dataset/ventilator_data.csv'
doc_path = 'https://raw.githubusercontent.com/Archerontheway/Summer_C_project/master/Data%20Source/medical_resource_dataset/medical_resource.csv'

ven_df = pd.read_csv(ven_path)
doc_df = pd.read_csv(doc_path)

medical_df = pd.merge(doc_df,ven_df,how = 'left', on = 'County')
medical_df

,County,Clinician_Count_MD,Clinician_Count_DO,Nurse_practitioner_Count,Available beds,Bed Census,Total Staffed Bed Capacity,Est Available Ventilators,County ICU Beds,County Staffed Beds,Hospitalized,ICU Cases,Latest COVID-19 Cases,Percent Remaining Vent Capacity,Ventilator Capacity
0,Alachua,2225,74,600,267.0,1397.0,1664.0,110.0,387,"2,263",13.0,4.0,716.0,0.990692,430
1,Baker,17,2,17,95.0,530.0,625.0,3.0,4,81,1.0,1.0,41.0,0.750000,4
2,Bay,347,27,164,109.0,414.0,523.0,49.0,77,676,4.0,1.0,190.0,0.986711,75
3,Bradford,17,1,18,0.0,0.0,0.0,3.0,4,25,2.0,1.0,61.0,0.750000,4
4,Brevard,1239,118,421,410.0,1153.0,1563.0,125.0,288,"1,867",15.0,5.0,850.0,0.983607,305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,9,0,7,43.0,82.0,125.0,1.0,25,145,1.0,1.0,41.0,0.966942,30
63,Volusia,935,92,297,400.0,1038.0,1438.0,77.0,278,"1,514",21.0,7.0,1218.0,0.977671,314
64,Wakulla,11,2,9,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,40.0,0.000000,0
65,Walton,97,16,37,53.0,64.0,117.0,10.0,28,112,4.0,1.0,181.0,0.967742,31


## about population

In [12]:
pop_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/population_data/pop_data.xlsx'

pop_df = pd.read_excel(pop_path)
pop_df

,County,overweight or obese(%),underweight(%),Population,land area(sq mi),population density,RaceWhite,RaceBlack,RaceOther,RaceUnknown,HispanicYES,HispanicNO,HispanicUnk
0,Alachua,56.9,2.0,263148,875.02,300.733698,439,240,214,326,224,651,344
1,Baker,72.5,1.4,27785,585.23,47.477060,54,14,2,2,1,67,4
2,Bay,66.3,2.3,182482,758.46,240.595417,286,44,23,37,97,243,50
3,Bradford,70.6,2.7,26979,293.96,91.777793,64,17,3,0,6,76,2
4,Brevard,62.9,3.7,576808,1015.66,567.914460,1159,129,127,531,349,1064,533
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,78.6,1.6,15239,243.56,62.567745,38,21,2,1,3,57,2
63,Volusia,65.8,3.5,527634,1101.03,479.218550,986,329,404,483,262,1264,676
64,Wakulla,73.3,1.3,31877,606.42,52.565878,38,17,3,5,5,53,5
65,Walton,61.6,2.1,65858,1037.62,63.470249,197,12,5,10,63,150,11


## about economics

In [13]:
eco_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/economic_dataset/economic_data.xlsx'
eco_df = pd.read_excel(eco_path)

eco_df

,County,GDP,income,FPLi,cases,death,lethality,perin,logG,Logperin,Logl,population,pergdp
0,Alachua,12447381,11983398,97.45,551,10,0.018149,44390,7.095078,4.647285,-1.741152,267306,46.566037
1,Baker,595832,916242,96.45,30,4,0.133333,32313,5.775124,4.509377,-0.875061,28249,21.092145
2,Bay,7606775,8010201,95.83,153,4,0.026144,43231,6.881201,4.635795,-1.582631,167283,45.472493
3,Bradford,659865,930593,95.83,52,2,0.038462,33557,5.819455,4.525783,-1.414973,28682,23.006241
4,Brevard,23437467,27112075,98.36,565,15,0.026549,45425,7.369911,4.657295,-1.575957,594469,39.425886
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,299208,332065,94.61,38,1,0.026316,22227,5.475973,4.346881,-1.579784,15505,19.297517
63,Volusia,16446375,23201619,96.00,946,48,0.050740,42374,7.216070,4.627099,-1.294650,538763,30.526178
64,Wakulla,606163,1201906,94.02,35,1,0.028571,37026,5.782589,4.568507,-1.544068,32976,18.381944
65,Walton,2511146,4263054,97.37,150,9,0.060000,59728,6.399872,4.776178,-1.221849,70071,35.837165


## about mobility

In [16]:
data_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/Mobility_data/applemobilitytrends_2020_06_29.xlsx'
mobility_data = pd.read_excel(data_path)
county_data = mobility_data[mobility_data['county'].str.contains('County')].reset_index(drop = True)

mobility_df = county_data.drop(columns = ['geo_type','sub-region','country'],axis = 1)
mobility_df['county'] = mobility_df['county'].apply(lambda x: x[:-7])
mobility_df

,county,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,...,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29
0,Alachua,driving,100,102.21,111.36,116.55,141.97,130.21,92.69,104.06,...,121.64,87.92,105.58,104.22,107.49,115.48,127.04,122.16,95.71,104.81
1,Baker,driving,100,106.73,104.79,115.21,160.07,136.27,107.89,124.29,...,215.68,161.19,159.74,152.54,158.22,178.65,209.67,212.64,206.44,170.63
2,Bay,driving,100,106.55,107.45,109.01,120.89,110.74,84.16,103.14,...,431.91,324.65,337.99,326.81,330.84,352.98,391.02,410.83,327.23,320.18
3,Bradford,driving,100,101.71,96.89,118.15,141.98,123.76,92.72,107.28,...,168.79,124.68,136.89,137.74,131.42,141.68,177.30,164.72,126.87,136.31
4,Brevard,driving,100,98.62,101.06,104.98,121.63,118.77,93.47,102.19,...,137.21,106.54,121.69,120.04,123.34,127.72,143.87,132.59,106.95,117.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Taylor,driving,100,103.05,95.17,126.25,157.55,144.24,105.58,129.29,...,217.32,213.09,160.74,168.25,166.84,193.98,243.79,226.02,196.65,169.81
57,Volusia,driving,100,102.07,103.37,108.03,130.87,131.98,100.69,106.19,...,173.77,135.30,137.50,135.81,142.67,148.29,169.83,174.03,138.89,136.67
58,Wakulla,driving,100,103.25,104.06,114.88,134.91,154.80,106.72,118.13,...,239.65,173.30,165.72,151.87,157.33,169.60,198.92,220.66,182.00,156.88
59,Walton,driving,100,105.32,112.34,125.07,160.33,147.91,128.84,138.46,...,649.62,453.29,447.90,457.11,439.10,463.27,486.81,632.70,472.59,427.71
